In [1]:
import os

import pickle

import numpy as np
import torch

import open3d as o3d

import matplotlib.pyplot as plt
import matplotlib.cm as cm
cmap = cm.get_cmap('jet')

from tqdm.notebook import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ipykernel_838641/1176535136.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('jet')


In [2]:
import ipywidgets as widgets
from IPython.display import display

In [3]:
PREPROCESSED_3D_DIR = "/home/rsl_admin/openscene/openscene/data/matterport_3d/test"

FUSED_FEATURES_DIR = "/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test"

OUTPUT_DIR = "/home/rsl_admin/openscene/comparison_outputs/segmentations"

In [4]:
if "openseg" in FUSED_FEATURES_DIR:
    embedding_space = 'openseg'
else:
    raise Exception("cannot determine the embedding space")
    
split = os.path.basename(PREPROCESSED_3D_DIR)

print(f"embedding space: {embedding_space}\nsplit: {split}")

embedding space: openseg
split: test


In [5]:
save_dirname = f"{embedding_space}-matterport-{split}"

os.makedirs(
    os.path.join(OUTPUT_DIR, save_dirname),
    exist_ok=True
)

print(f"saving to: {os.path.join(OUTPUT_DIR, save_dirname)}")

saving to: /home/rsl_admin/openscene/comparison_outputs/segmentations/openseg-matterport-test


##
First check that every 3D region has fused features for that region

In [6]:
import re

In [7]:
data_3d_fnames = os.listdir(PREPROCESSED_3D_DIR)
fused_feats_fnames = []

for fname in data_3d_fnames:
    scan_name = fname.split("_")[0]
    
    match = re.search(
        r'_region(\d+)\.pth', 
        fname
    )
    if match:
        region_number = match.group(1)
    else:
        raise RuntimeError(f"No region number for file {fname}")

    fused_feats_fname = f"{scan_name}_region{region_number}_0.pt"
    
    if not os.path.isfile(
        os.path.join(FUSED_FEATURES_DIR, fused_feats_fname)
    ):
        raise Exception(f"3D data file {fname} has no corresponding fused feature file {fused_feats_fname}")
                        
    fused_feats_fnames.append(fused_feats_fname)

In [8]:
# for x, y in zip(data_3d_fnames, fused_feats_fnames):
#     print(f"{x} - {y}")

## Setup 3D model

In [9]:
'''simple config class to recreate what's done in the openscene code'''
class ModelConfig:
    def __init__(self, feature_2d_extractor, arch_3d):
        self.feature_2d_extractor = feature_2d_extractor
        self.arch_3d = arch_3d

In [10]:
if embedding_space == "openseg":
    checkpoint_path = "/home/rsl_admin/openscene/checkpoints/matterport_openseg.pth"
else:
    raise NotImplementedError

checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage.cuda())

In [11]:
from run.distill import get_model

model_cfg = ModelConfig(
    feature_2d_extractor=embedding_space, 
    arch_3d='MinkUNet18A',
)
model = get_model(model_cfg)
model.load_state_dict(checkpoint['state_dict'], strict=True)

model = model.cuda()

In [12]:
from comparison_utils import DisNetRunner

In [13]:
disnet_runner = DisNetRunner(model)

### Set up class vocabulary
This step requires the user to select a label set to query the segmentations on

In [14]:
labelset_selection_dropdown  = widgets.Dropdown(
    options=[
        "MATTERPORT_LABELS_21",
        "MATTERPORT_LABELS_REGIONS",
    ],
    disabled=False,
)

display(labelset_selection_dropdown)

Dropdown(options=('MATTERPORT_LABELS_21', 'MATTERPORT_LABELS_REGIONS'), value='MATTERPORT_LABELS_21')

In [15]:
if labelset_selection_dropdown.value == "MATTERPORT_LABELS_21":
    from dataset.label_constants import MATTERPORT_LABELS_21
    labelset = list(MATTERPORT_LABELS_21)
    use_prompt_eng = True

elif labelset_selection_dropdown.value == "MATTERPORT_LABELS_REGIONS":
    from dataset.label_constants import MATTERPORT_LABELS_REGIONS
    labelset = list(MATTERPORT_LABELS_REGIONS)
    
    # Not sure if should use prompt engineering for regions or not, 
    # maybe run with and without and see which one is better?
    use_prompt_eng = True

else: 
    raise Exception


In [16]:
label_to_ind = {label: i for i, label in enumerate(labelset)}

print(label_to_ind)

{'bathroom': 0, 'bedroom': 1, 'closet': 2, 'dining room': 3, 'garage': 4, 'hallway': 5, 'library': 6, 'laundry room or mudroom': 7, 'kitchen': 8, 'living room': 9, 'meeting room or conference room': 10, 'office': 11, 'porch or terrace or deck or driveway': 12, 'rec/game': 13, 'stairs': 14, 'utility room or tool room': 15, 'cinema or home theater or theater"': 16, 'gym': 17, 'balcony': 18, 'bar': 19, 'classroom': 20, 'spa or sauna': 21, 'other room': 22, 'junk': 23, 'no label': 24, 'dining booth': 25, 'entryway/foyer/lobby': 26, 'outdoor': 27}


In [17]:
from comparison_utils import extract_text_feature

In [18]:
label_feats = extract_text_feature(
    labelset,
    prompt_eng=use_prompt_eng
)

print(label_feats.shape)

Use prompt engineering: a XX in a scene
Loading CLIP ViT-L/14@336px model...
Finish loading
torch.Size([28, 768])


In [19]:
from comparison_utils import get_region_features, compute_predictions

In [20]:
for data_3d_fname, fused_feats_fname in tqdm(
    zip(data_3d_fnames, fused_feats_fnames), total=len(data_3d_fnames)
):
    
#     print(data_3d_fname)
#     print(fused_feats_fname)
    
    points, colors, fused_feats, distill_feats = get_region_features(
        os.path.join(PREPROCESSED_3D_DIR, data_3d_fname),
        os.path.join(FUSED_FEATURES_DIR, fused_feats_fname),
        disnet_runner,
    )
    
    preds = compute_predictions(
        label_feats,
        fused_feats,
        distill_feats,
        method="ensemble"
    )
    
    out = {
        "points": points.astype(np.float16),
        "colors": colors.astype(np.float16),
        "preds": preds.astype(np.int16),
        "label_to_ind": label_to_ind,
    }
    
    save_path = os.path.join(
        os.path.join(OUTPUT_DIR, save_dirname, data_3d_fname.replace('.pth', '.pkl')),
    )
    
    with open(save_path, 'wb') as f:
        pickle.dump(out, f)

  0%|          | 0/406 [00:00<?, ?it/s]

## Visualize an output segmentation

In [21]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

In [22]:
np.random.seed(0)

In [23]:
# output_fname = np.random.choice(
#     os.listdir(os.path.join(OUTPUT_DIR, save_dirname))
# )

output_fname = "2t7WUuJeko7_region0.pkl"

output_path = os.path.join(OUTPUT_DIR, save_dirname, output_fname)

with open(output_path, 'rb') as f:
    out = pickle.load(f)
    
print(f"loaded {output_fname}")

loaded 2t7WUuJeko7_region0.pkl


In [24]:
scan_name = output_fname.split('_')[0]

scan_mesh = o3d.io.read_triangle_mesh(
    os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}/house_segmentations/{scan_name}.ply")
)

In [25]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(out["points"])
pcd.colors = o3d.utility.Vector3dVector(out["colors"])

In [26]:
label_to_ind = out["label_to_ind"]
labels = label_to_ind.keys()

In [27]:
print(labels)

dict_keys(['bathroom', 'bedroom', 'closet', 'dining room', 'garage', 'hallway', 'library', 'laundry room or mudroom', 'kitchen', 'living room', 'meeting room or conference room', 'office', 'porch or terrace or deck or driveway', 'rec/game', 'stairs', 'utility room or tool room', 'cinema or home theater or theater"', 'gym', 'balcony', 'bar', 'classroom', 'spa or sauna', 'other room', 'junk', 'no label', 'dining booth', 'entryway/foyer/lobby', 'outdoor'])


In [28]:
label_dropdown = widgets.Dropdown(
    options=['all'] + sorted(labels),
    disabled=False,
)

display(label_dropdown)

Dropdown(options=('all', 'balcony', 'bar', 'bathroom', 'bedroom', 'cinema or home theater or theater"', 'class…

In [39]:
label_inds = np.where(out["preds"] == label_to_ind[label_dropdown.value])[0]

o3d.visualization.draw_geometries(
    [
        scan_mesh, 
        pcd.select_by_index(label_inds).paint_uniform_color((0,1,0))
    ]
)